In [ ]:
from entsoe import EntsoePandasClient
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Setup client
client = EntsoePandasClient(api_key='1206c24a-2fb6-4ec3-a47f-0e1be10dacd4')  

# Define zone: for example, Netherlands is 'NL'
country_code = 'NL'

# Define time period
start = pd.Timestamp("2017-01-01 00:00", tz='Europe/Amsterdam')
end   = pd.Timestamp("2019-04-01 00:00", tz='Europe/Amsterdam')


In [3]:
day_ahead_price = client.query_day_ahead_prices(country_code, start=start, end=end)
load_forecast = client.query_load_forecast(country_code, start=start, end=end)
wind_forecast = client.query_wind_and_solar_forecast(country_code, start=start, end=end)


In [4]:
# Combine wind forecasts
wind_forecast_series = (
    wind_forecast['Wind Onshore'] + wind_forecast['Wind Offshore']
)

# Extract 1D series from load forecast
load_forecast_series = load_forecast.iloc[:, 0]

# Construct final DataFrame using 1D Series only
df = pd.DataFrame({
    'Price': day_ahead_price.squeeze(),
    'Grid load forecast': load_forecast_series,
    'Wind power forecast': wind_forecast_series,
}).reset_index().rename(columns={'index': 'Date'})

df = df.resample('h', on='Date').mean()


In [5]:
# If the index is a datetime index, reset it
if df.index.name == 'Date' or df.index.name is not None:
    df = df.reset_index()

# Make sure the column is named 'Date'
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)



In [6]:
print(df.head())
print(df.tail())
print(df[df['Date'] == '2017-03-26 03:00:00'])

                 Date  Price  Grid load forecast  Wind power forecast
0 2017-01-01 00:00:00  42.00            11186.00              1456.75
1 2017-01-01 01:00:00  49.94            10492.75              1502.25
2 2017-01-01 02:00:00  51.95             9893.75              1530.75
3 2017-01-01 03:00:00  40.99             9529.75              1568.50
4 2017-01-01 04:00:00  39.00             9477.25              1580.00
                     Date  Price  Grid load forecast  Wind power forecast
19675 2019-03-31 20:00:00  41.80            10318.00              2046.25
19676 2019-03-31 21:00:00  39.08            10105.75              2065.00
19677 2019-03-31 22:00:00  40.30             9632.50              2024.25
19678 2019-03-31 23:00:00  37.51             9007.75              1887.25
19679 2019-04-01 00:00:00  31.06                 NaN                  NaN
                    Date  Price  Grid load forecast  Wind power forecast
2018 2017-03-26 03:00:00  26.97            10178.75            

In [7]:
# Ensure datetime and sorted
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Remove duplicates
df = df.drop_duplicates('Date', keep='first')  # or .groupby(...)

# Reindex
df = df.set_index('Date')
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
df = df.reindex(full_range)

# Fill missing
df = df.ffill()

# Reset index
df.index.name = 'Date'
df = df.reset_index()

In [8]:
from pathlib import Path

# Define the path one level up and into 'datasets'
output_path = Path("..") / "datasets" / "2019NLDAP.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)

# Save CSV
df.to_csv(output_path, index=False)
